In [1]:
from astropy.io import fits
import sys
import glob
import numpy
from desitarget.sv1.sv1_targetmask import bgs_mask as bgs_mask_sv1
from desitarget.sv2.sv2_targetmask import bgs_mask as bgs_mask_sv2
from desitarget.sv3.sv3_targetmask import bgs_mask as bgs_mask_sv3


from desispec.io import read_spectra, write_spectra
from desispec.spectra import Spectra
from desispec.coaddition import coadd_cameras
from astropy.table import Table, Column, vstack, hstack, join
from desiutil.log import get_logger, DEBUG, ERROR

In [2]:
log = get_logger(ERROR)

In [3]:
# We may not need this.  We are omitting SV1 and the top-level masks of the rest
# are consistent.
sv1_bgs_bits = '|'.join([_ for _ in bgs_mask_sv1.names() if 'BGS' in _])
sv2_bgs_bits = '|'.join([_ for _ in bgs_mask_sv2.names() if 'BGS' in _])
sv3_bgs_bits = '|'.join([_ for _ in bgs_mask_sv3.names() if 'BGS' in _])

### TARGET bitmasks
It has been confirmed that each spectrum is uniquely `SV1`, `SV2`, `SV3`, or Main Survey.  We thus compress these columns into one.

    desi=> select  count(*) from everest.zpix_redshifts where (SV2_DESI_TARGET !=0 or SV2_MWS_TARGET !=0 or SV2_BGS_TARGET !=0 or SV2_SCND_TARGET !=0) AND (DESI_TARGET !=0 or MWS_TARGET !=0 or BGS_TARGET !=0);
     count 
    -------
         0
    (1 row)

In [4]:
def getTARGET(fibermap):
    
    targets = ["DESI_TARGET", "MWS_TARGET", "BGS_TARGET", "SCND_TARGET"]
    
    ans=[]
    for target in targets:
        colnames=[dum for dum in fibermap.columns.keys() if target in dum]
        if len(colnames) == 0:
            desi_target = Column(numpy.zeros(len(fibermap)), name=target, dtype=numpy.int64)
        else:
            desi_target = fibermap[colnames[0]]
            dum = fibermap[colnames[0]] !=0
            for i in range(1,len(colnames)):
                desi_target = desi_target+ fibermap[colnames[i]]
                dum = dum + fibermap[colnames[i]] !=0

            if max(dum)>1:
                raise RuntimeError(target+": unexpected behavior as only one of these is expected")

        ans.append(desi_target)

    return ans

## Everest

There are three types of data, `perexp`, `pernight`, and `cumulative`.  Each has its own subdirectory structure.  Note that the contents of the files may be identical depending if there is only one
exposure.

### Spectra
While spectra are saved in different files, the `coadd*.fits` provide a common format for the spectra we want from all three subdirectories.

HDUs inside the fits file are 'FIBERMAP', 'EXP_FIBERMAP', 'X_WAVELENGTH', 'X_FLUX', 'X_IVAR', 'X_MASK', 'X_RESOLUTION', 'SCORES' where X \in B, R, Z

FIBERMAP contains metadata that we want to save.



### Redshifts
Redshift information is in redrock files.

HDUs inside the fits file are 'REDSHIFTS', 'FIBERMAP', 'EXP_FIBERMAP', 'TSNR2'

### SV's
The bitmasks for SV1 are consistent with others so SV1 targets are omitted

In [5]:
rootdir="/global/project/projectdirs/desi/spectro/redux/everest/tiles/"
subdirs=["perexp", "pernight","cumulative"]
subdir=subdirs[1]
allcoadd_files = glob.glob(rootdir+subdir+'/*/*/coadd-*-*-*.fits')

In [ ]:
for name in allcoadd_files:
    
    #scrub path for information
    dum = name.split('/')
    sd1 = dum[-3]
    sd2 = dum[-2]
    subname = name[name.find('coadd')+6:-5]
    dum = subname.split('-')
    panel=dum[0]
    
    zname=name.replace('coadd','redrock')
    
    #Read spectra and coadd cameras, read 
    
    pspectra = read_spectra(name)
    cspectra = coadd_cameras(pspectra)
    zbest = Table.read(zname, 'redshifts')

    #Make selection: only targets in a good fiber

    select = (cspectra.fibermap['OBJTYPE'] == 'TGT') & (cspectra.fibermap['COADD_FIBERSTATUS'] == 0)
    
    # Also select targets not from SV1.  SV1 targets have a non-zero
    # value in a column SV1_*_TARGET
    sv1_colnames=[dum for dum in cspectra.fibermap.columns.keys() if "SV1_" in dum]
    if len(sv1_colnames) !=0:
        notinSV1 = cspectra.fibermap[sv1_colnames[0]] ==0
        for i in range(1,len(sv1_colnames)):
            notinSV1 = numpy.logical_and(notinSV1, cspectra.fibermap[sv1_colnames[i]] ==0)
        select = numpy.logical_and(select,notinSV1)

    # if there is nothing selected move on to the next file
    if select.sum()==0:
        continue

    #Access the relevant keys
    
    tid = cspectra.fibermap[select]['TARGETID']
    tra = cspectra.fibermap[select]['TARGET_RA']
    tdec = cspectra.fibermap[select]['TARGET_DEC']

    desi_target, mws_target, bgs_target, scnd_target = getTARGET(cspectra.fibermap[select])
    
    zs = zbest[select]['Z'],
    zerr = zbest[select]['ZERR']
    zwarn = zbest[select]['ZWARN']
    spectype = zbest[select]['SPECTYPE']
    subtype = zbest[select]['SUBTYPE']
    deltachi2 = zbest[select]['DELTACHI2']
        
    
    # Access spectra here
    
    allwave = cspectra.wave['brz']
    allflux = cspectra.flux['brz'][select]
    allivar = cspectra.ivar['brz'][select]
    allmask = cspectra.mask['brz'][select]

INFO:spectra.py:282:read_spectra: iotime 0.453 sec to read coadd-4-338-20210414.fits at 2021-12-22T11:42:00.876355
INFO:spectra.py:282:read_spectra: iotime 0.453 sec to read coadd-8-338-20210414.fits at 2021-12-22T11:42:04.216889
INFO:spectra.py:282:read_spectra: iotime 0.455 sec to read coadd-3-338-20210414.fits at 2021-12-22T11:42:07.361006
INFO:spectra.py:282:read_spectra: iotime 0.419 sec to read coadd-2-338-20210414.fits at 2021-12-22T11:42:10.366423
INFO:spectra.py:282:read_spectra: iotime 0.429 sec to read coadd-5-338-20210414.fits at 2021-12-22T11:42:13.442141
INFO:spectra.py:282:read_spectra: iotime 0.454 sec to read coadd-9-338-20210414.fits at 2021-12-22T11:42:16.496221
INFO:spectra.py:282:read_spectra: iotime 0.414 sec to read coadd-6-338-20210414.fits at 2021-12-22T11:42:19.795866
INFO:spectra.py:282:read_spectra: iotime 0.411 sec to read coadd-7-338-20210414.fits at 2021-12-22T11:42:22.720412
INFO:spectra.py:282:read_spectra: iotime 0.384 sec to read coadd-0-338-20210414.